In [11]:
## Hi There!

# This is just a personal project that I worked on. Was keen on doing some data analysis using python and then attempting to predict the Euro 2024 scores.
# My Goals:
# 1. Predict at least 50% of the group stage winners
# 2. Have the predicted winner make it to at least the QF

### Extracting tables from the upcoming Euros

# Libraries
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle
from bs4 import BeautifulSoup
import requests
from scipy.stats import poisson

# Import the wikipedia page:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/UEFA_Euro_2024')

# Understanding the correct tables:
all_tables[18]
all_tables[25]
all_tables[32]
all_tables[39]
all_tables[46]
all_tables[53]

# Create dictionary + assign the group letters to the groups
dict_table = {}
for letter, i in zip(alphabet, range(18, 60, 7)):
    df = all_tables[i]
    # Rename from "Teamvte" to Team
    df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
    df['Team'] = df['Team'].str.replace(' (H)', '', regex=True).str.strip()
    # Remove the qualification column from the group
    if 'Qualification' in df.columns:
        df.pop("Qualification")
    # Initialize 'Pts' column
    df['Pts'] = 0
    dict_table[f'Group {letter}'] = df

# Debugging step: Check the columns of each DataFrame in dict_table
for group in dict_table:
    print(f"Columns in {group}: {dict_table[group].columns}")

### Extracting Football Matches
years = [1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]

def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/UEFA_Euro_{year}'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')

    matches = soup.find_all('div', class_= 'footballbox')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football

euro = [get_matches(year) for year in years]
df_euro = pd.concat(euro, ignore_index=True)
df_euro.to_csv('euro_historical_data.csv', index=False)

# Fixtures for 2024
df_fixture = get_matches(2024)
df_fixture.to_csv('euro_2024_fixtures.csv', index=False)

### Data Cleaning and Transformation
df_historical_data = pd.read_csv('euro_historical_data.csv')
df_fixture = pd.read_csv('euro_2024_fixtures.csv')

# Cleaning


df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

# Getting rid of the (a.e.t)
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)


# Cleaning the scores and giving them to home and away
df_historical_data[['Home Goal', 'Away Goal']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

#TEST

df_historical_data['home'] = df_historical_data['home'].str.replace('Germany', 'Germany (H)', regex=True)
df_historical_data['away'] = df_historical_data['away'].str.replace('Germany', 'Germany (H)', regex=True)

# Modify the team names in the dict_table to include (H)
for group, df in dict_table.items():
    df['Team'] = df['Team'].replace('Germany', 'Germany (H)')

# Verify the modified dict_table
for group, df in dict_table.items():
    print(f"{group}:\n{df}\n")

#END TEST
# Renaming Columns + Converting goals from object to int
df_historical_data.rename(columns={'home': 'Home Team', 'away': 'Away Team', 'year': 'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'Home Goal': int, 'Away Goal': int, 'Year': int})

# Creating new column for total goals
df_historical_data['Total Goals'] = df_historical_data['Home Goal'] + df_historical_data['Away Goal']

### Exporting the Cleaned DF
df_historical_data.to_csv('cleaned_euro_historical_data.csv', index=False)
df_fixture.to_csv('cleaned_euro_2024_fixtures.csv', index=False)

### Time to build the model
dict_table = pickle.load(open('dict_table', 'rb'))
df_historical_data = pd.read_csv('cleaned_euro_historical_data.csv')
df_fixture = pd.read_csv('cleaned_euro_2024_fixtures.csv')

# Split df into df_home and df_away
df_home = df_historical_data[['Home Team', 'Home Goal', 'Away Goal']]
df_away = df_historical_data[['Away Team', 'Home Goal', 'Away Goal']]
df_home = df_home.rename(columns={'Home Team': 'Team', 'Home Goal': 'Goals Scored', 'Away Goal': 'Goals Conceded'})
df_away = df_away.rename(columns={'Away Team': 'Team', 'Home Goal': 'Goals Conceded', 'Away Goal': 'Goals Scored'})

# Build the team strength
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
# Calculating Team Strength
# Split df into df_home and df_away
df_home = df_historical_data[['Home Team', 'Home Goal', 'Away Goal']]
df_away = df_historical_data[['Away Team', 'Home Goal', 'Away Goal']]
df_home = df_home.rename(columns={'Home Team': 'Team', 'Home Goal': 'Goals Scored', 'Away Goal': 'Goals Conceded'})
df_away = df_away.rename(columns={'Away Team': 'Team', 'Home Goal': 'Goals Conceded', 'Away Goal': 'Goals Scored'})

# Building the team strength
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

# Prediction
# Using Poisson distribution 
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # Goals scored * goals conceded
        lamb_home = df_team_strength.at[home, 'Goals Scored'] * df_team_strength.at[away, 'Goals Conceded']
        lamb_away = df_team_strength.at[away, 'Goals Scored'] * df_team_strength.at[home, 'Goals Conceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0, 11):
            for y in range(0, 11):
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

# Predictions
df_fixture_group36 = df_fixture[:36].copy()
df_fixture_knockout = df_fixture[36:44].copy()
df_fixture_quarter = df_fixture[44:48].copy()
df_fixture_semi = df_fixture[48:50].copy()
df_fixture_final = df_fixture[50:].copy()

# Running all the games in the group stage
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group36[df_fixture_group36['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)



Columns in Group A: Index(['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'], dtype='object')
Columns in Group B: Index(['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'], dtype='object')
Columns in Group C: Index(['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'], dtype='object')
Columns in Group D: Index(['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'], dtype='object')
Columns in Group E: Index(['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'], dtype='object')
Columns in Group F: Index(['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'], dtype='object')
Group A:
   Pos         Team  Pld  W  D  L  GF  GA  GD  Pts
0    1  Germany (H)    0  0  0  0   0   0   0    0
1    2     Scotland    0  0  0  0   0   0   0    0
2    3      Hungary    0  0  0  0   0   0   0    0
3    4  Switzerland    0  0  0  0   0   0   0    0

Group B:
   Pos     Team  Pld  W  D  L  GF  GA  GD  Pts
0    1    Spain    0  0  0  0   0   0   0  

C:\Users\Prathamesh\AppData\Local\Temp\ipykernel_17200\2427201678.py:186: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.43618806]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Prathamesh\AppData\Local\Temp\ipykernel_17200\2427201678.py:186: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.79333433]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Prathamesh\AppData\Local\Temp\ipykernel_17200\2427201678.py:186: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.34102659]' has dtype inc

In [10]:
from tabulate import tabulate

# Iterate over each group in the dict_table
for group, df in dict_table.items():
    print(f"Group {group}:")
    print(tabulate(df, headers='keys', tablefmt='psql'))  # Use 'psql' format for a nice table
    print()  # Add a newline for better readability


Group Group A:
+----+-------------+-------+
|    | Team        |   Pts |
|----+-------------+-------|
|  0 | Hungary     |     3 |
|  1 | Switzerland |     3 |
|  2 | Scotland    |     2 |
|  3 | Germany (H) |     0 |
+----+-------------+-------+

Group Group B:
+----+---------+-------+
|    | Team    |   Pts |
|----+---------+-------|
|  0 | Spain   |     5 |
|  1 | Italy   |     5 |
|  2 | Croatia |     4 |
|  3 | Albania |     2 |
+----+---------+-------+

Group Group C:
+----+----------+-------+
|    | Team     |   Pts |
|----+----------+-------|
|  0 | England  |     4 |
|  1 | Denmark  |     2 |
|  2 | Slovenia |     2 |
|  3 | Serbia   |     0 |
+----+----------+-------+

Group Group D:
+----+-------------+-------+
|    | Team        |   Pts |
|----+-------------+-------|
|  0 | Netherlands |     6 |
|  1 | France      |     5 |
|  2 | Poland      |     3 |
|  3 | Austria     |     2 |
+----+-------------+-------+

Group Group E:
+----+----------+-------+
|    | Team     |   Pts

In [12]:
df_fixture_knockout

,home,score,away,year
36,Runner-up Group A,Match 38,Runner-up Group B,2024
37,Winner Group A,Match 37,Runner-up Group C,2024
38,Winner Group C,Match 40,3rd Group D/E/F,2024
39,Winner Group B,Match 39,3rd Group A/D/E/F,2024
40,Runner-up Group D,Match 42,Runner-up Group E,2024
41,Winner Group F,Match 41,3rd Group A/B/C,2024
42,Winner Group E,Match 43,3rd Group A/B/C/D,2024
43,Winner Group D,Match 44,Runner-up Group F,2024


In [13]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
36,Runner-up Group A,Match 38,Runner-up Group B,2024,?
37,Winner Group A,Match 37,Runner-up Group C,2024,?
38,Winner Group C,Match 40,3rd Group D/E/F,2024,?
39,Winner Group B,Match 39,3rd Group A/D/E/F,2024,?
40,Runner-up Group D,Match 42,Runner-up Group E,2024,?
41,Winner Group F,Match 41,3rd Group A/B/C,2024,?
42,Winner Group E,Match 43,3rd Group A/B/C/D,2024,?
43,Winner Group D,Match 44,Runner-up Group F,2024,?


In [14]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [40]:
get_winner(df_fixture_knockout)


,home,score,away,year,winner
37,Winner Group A,Match 37,Runner-up Group C,2024,Runner-up Group C
38,Winner Group C,Match 40,3rd Group D/E/F,2024,3rd Group D/E/F
39,Winner Group B,Match 39,3rd Group A/D/E/F,2024,3rd Group A/D/E/F
40,Runner-up Group D,Match 42,Runner-up Group E,2024,Runner-up Group E
41,Winner Group F,Match 41,3rd Group A/B/C,2024,3rd Group A/B/C
42,Winner Group E,Match 43,3rd Group A/B/C/D,2024,3rd Group A/B/C/D
43,Winner Group D,Match 44,Runner-up Group F,2024,Runner-up Group F
